In [1]:
!pip install pandas numpy scikit-learn surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357284 sha256=08face241155e71eb7569e97a280a624aa16e4491a16a9ba9df5f5bf475bb1ac
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
import pandas as pd

# Load the books dataset (bookId, title, authors, genres)
books = pd.read_csv('/content/books.csv')

# Load the ratings dataset (userId, bookId, rating)
ratings = pd.read_csv('/content/ratings.csv')


In [5]:
# Clean columns
ratings.columns = ratings.columns.str.strip()

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd

# Load your data (adjust the path as needed)
ratings = pd.read_csv("ratings.csv")  # Ensure columns: user_id, book_id, rating
books = pd.read_csv("books.csv")  # Ensure columns: bookId, title

# Create a sparse user-item interaction matrix
interaction_matrix = ratings.pivot(index='user_id', columns='book_id', values='rating').fillna(0)
sparse_interaction_matrix = csr_matrix(interaction_matrix.values)

# Function to recommend books based on user similarity (computes similarity on demand)
def recommend_books(user_id, top_n=5):
    user_idx = user_id - 1  # Adjusting for zero-based indexing
    user_vector = sparse_interaction_matrix[user_idx]

    # Calculate cosine similarity with other users (only with non-zero rows to save memory)
    similarity = cosine_similarity(user_vector, sparse_interaction_matrix)[0]

    # Get indices of top similar users, excluding the target user
    similar_user_indices = np.argsort(similarity)[-top_n-1:-1][::-1]

    # Collect book recommendations from top similar users
    recommended_books = []
    for idx in similar_user_indices:
        user_ratings = interaction_matrix.iloc[idx]
        recommended_books.extend(user_ratings[user_ratings > 0].index.tolist())

    # Remove duplicates and limit to top N recommendations
    recommended_books = list(set(recommended_books))[:top_n]

    # Map book IDs to book titles
    book_titles = books[books['book_id'].isin(recommended_books)]['title'].values
    return book_titles



In [2]:
# Example: Recommend top 5 books for user with ID 1
recommended_books = recommend_books(1, top_n=5)
print("Recommended Books for User 1:")
for book in recommended_books:
    print(book)


Recommended Books for User 1:
The Hunger Games (The Hunger Games, #1)
Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
Twilight (Twilight, #1)
To Kill a Mockingbird
The Great Gatsby
